In [160]:
!pip -q install -U scikit-learn pandas numpy openpyxl xarray netCDF4 matplotlib plotly


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [171]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from calendar import monthrange
import xarray as xr
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"  # ou "vscode", "jupyterlab", etc.

## Extração dos Dados

### Download dos dados a partir do earthdata

Para utilizar devidamente esse script é necessário se cadastrar em https://urs.earthdata.nasa.gov, após isso, crie um .env com suas credenciais para fazer o download desse dataset, ajuste as variáveis iniciais de acordo com o folder desejado e range da data de download. Foi implementado paralelismo para acelerar o processo de download, ajuste conforme necessário.

In [266]:
import os
import sys
import time
import netrc
import pathlib
import platform
from datetime import datetime, timedelta
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Iterable, Tuple, List, Optional

import requests


START = "2015-01-01"     # YYYY-MM-DD
END   = "2016-12-31"     # YYYY-MM-DD (inclusivo)
OUTDIR = "./dados_gpcp"  # pasta de saída
WORKERS = 4              # paralelismo
SKIP_EXISTING = True     # pular arquivos já existentes

# -------------------- Constantes do produto --------------------
EARTHDATA_HOST = "urs.earthdata.nasa.gov"
PRODUCT_ROOT = "https://data.gesdisc.earthdata.nasa.gov/data/GPCP/GPCPDAY.3.3"
VERSION_TAG = "V3.3"  # versão do produto

# --- Loader .env (zero-deps) ---
def load_dotenv_simple(path: str = ".env") -> None:
    p = pathlib.Path(path)
    if not p.exists():
        print(f"[aviso] .env não encontrado em {p.resolve()}")
        return
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        os.environ.setdefault(k.strip(), v.strip().strip('"').strip("'"))

load_dotenv_simple(".env")

# -------------------- Helpers --------------------
def get_credentials() -> Tuple[Optional[str], Optional[str]]:
    """Tenta obter (user, pwd) de env vars -> NETRC explícito -> _netrc/.netrc."""
    user = os.getenv("EARTHDATA_USERNAME")
    pwd  = os.getenv("EARTHDATA_PASSWORD")
    if user and pwd:
        return user, pwd

    netrc_env = os.getenv("NETRC")
    if netrc_env and pathlib.Path(netrc_env).exists():
        try:
            a = netrc.netrc(netrc_env)
            c = a.authenticators(EARTHDATA_HOST)
            if c and len(c) >= 3:
                return c[0], c[2]
        except Exception:
            pass

    home = pathlib.Path.home()
    candidates = [home / ".netrc"]
    if platform.system().lower().startswith("win"):
        candidates.insert(0, home / "_netrc")

    for p in candidates:
        if p.exists():
            try:
                a = netrc.netrc(str(p))
                c = a.authenticators(EARTHDATA_HOST)
                if c and len(c) >= 3:
                    return c[0], c[2]
            except Exception:
                continue

    return None, None

def human(n: float) -> str:
    for unit in ['','K','M','G','T','P']:
        if abs(n) < 1024.0:
            return f"{n:3.1f}{unit}B"
        n /= 1024.0
    return f"{n:.1f}EB"

def filename_from_url(url: str) -> str:
    return pathlib.Path(urlparse(url).path).name or "download.bin"

def daterange(start: datetime, end: datetime) -> Iterable[datetime]:
    cur = start
    step = timedelta(days=1)
    while cur <= end:
        yield cur
        cur += step

def build_gpcp_url(day: datetime) -> str:
    yyyy = day.strftime("%Y")
    ymd  = day.strftime("%Y%m%d")
    fname = f"GPCPDAY_L3_{ymd}_{VERSION_TAG}.nc4"
    return f"{PRODUCT_ROOT}/{yyyy}/{fname}"

def session_with_auth(user: Optional[str]=None, pwd: Optional[str]=None, token: Optional[str]=None) -> requests.Session:
    """Cria sessão; se houver token, usa Bearer; senão, usa Basic (user,pwd)."""
    s = requests.Session()
    s.headers.update({"User-Agent": "iDEV-downloader/1.3 (Notebook)"})
    if token:
        s.headers["Authorization"] = f"Bearer {token}"
    elif user and pwd:
        s.auth = (user, pwd)
    return s

def earthdata_fetch(session: requests.Session, url: str, auth_pair: Tuple[Optional[str], Optional[str]], token: Optional[str]=None, timeout=60) -> requests.Response:
    """
    Faz GET já com a auth disponível.
    Se 401 apontar para fluxo OAuth do URS, sugere consent/login no navegador.
    """
    user, pwd = auth_pair
    if token:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout)
    elif user and pwd:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout, auth=(user, pwd))
    else:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout)

    if r.status_code == 401 and "urs.earthdata.nasa.gov/oauth/authorize" in r.url:
        raise RuntimeError(
            "HTTP 401 via URS/OAuth. Se estiver usando token, verifique validade/escopo. "
            "Caso seja o primeiro acesso, abra a URL do dataset no navegador, faça login e autorize o domínio "
            "(consent) para sua conta Earthdata. Depois rode novamente."
        )
    return r

def download_one(url: str, outdir: pathlib.Path, skip_existing: bool, retries=3, backoff=2.0) -> pathlib.Path:
    outdir.mkdir(parents=True, exist_ok=True)
    outpath = outdir / filename_from_url(url)

    if skip_existing and outpath.exists() and outpath.stat().st_size > 0:
        print(f"[skip] {outpath.name} ({human(outpath.stat().st_size)})")
        return outpath

    user, pwd = get_credentials()
    token = os.getenv("EARTHDATA_TOKEN")
    s = session_with_auth(user, pwd, token)

    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = earthdata_fetch(s, url, (user, pwd), token=token)
            if r.status_code == 404:
                raise FileNotFoundError(f"HTTP 404 (não encontrado): {url}")
            if r.status_code != 200:
                raise RuntimeError(f"HTTP {r.status_code} – URL final: {r.url}")

            total = int(r.headers.get("Content-Length", 0))
            downloaded = 0
            start = time.time()

            tmp_path = outpath.with_suffix(outpath.suffix + ".part")
            with open(tmp_path, "wb") as f:
                for chunk_bytes in r.iter_content(chunk_size=1024*1024):
                    if not chunk_bytes:
                        continue
                    f.write(chunk_bytes)
                    downloaded += len(chunk_bytes)
                    if total > 0:
                        pct = 100 * downloaded / total
                        sys.stdout.write(f"\rBaixando {outpath.name}: {human(downloaded)}/{human(total)} ({pct:5.1f}%)")
                    else:
                        sys.stdout.write(f"\rBaixando {outpath.name}: {human(downloaded)}")
                    sys.stdout.flush()

            tmp_path.replace(outpath)
            dur = time.time() - start
            sys.stdout.write(f"\n[ok] {outpath.name} em {dur:.1f}s ({human(downloaded)})\n")
            return outpath

        except Exception as e:
            last_err = e
            if attempt < retries:
                wait = backoff ** (attempt - 1)
                print(f"[tentativa {attempt}/{retries} falhou] {e} → retry em {wait:.1f}s")
                time.sleep(wait)
            else:
                print(f"[erro] {e}")

    raise RuntimeError(f"Falha definitiva ao baixar: {url} → {last_err}")

# -------------------- Execução --------------------
try:
    start_dt = datetime.strptime(START, "%Y-%m-%d")
    end_dt   = datetime.strptime(END,   "%Y-%m-%d")
except ValueError as e:
    raise SystemExit(f"[erro] Datas inválidas: {e}\nUse o formato YYYY-MM-DD.")

if end_dt < start_dt:
    raise SystemExit("[erro] END não pode ser anterior a START.")

urls = [build_gpcp_url(d) for d in daterange(start_dt, end_dt)]
print(f"Total de arquivos a baixar: {len(urls)}")

outdir_path = pathlib.Path(OUTDIR).resolve()
failed: List[str] = []

if WORKERS <= 1:
    for u in urls:
        try:
            download_one(u, outdir_path, skip_existing=SKIP_EXISTING)
        except Exception as e:
            failed.append(f"{u} → {e}")
else:
    with ThreadPoolExecutor(max_workers=WORKERS) as pool:
        futs = {pool.submit(download_one, u, outdir_path, SKIP_EXISTING): u for u in urls}
        for fut in as_completed(futs):
            try:
                fut.result()
            except Exception as e:
                failed.append(f"{futs[fut]} → {e}")

print("\nConcluído.")
if failed:
    print("\nFalhas:")
    for item in failed:
        print(" -", item)


Total de arquivos a baixar: 731
Baixando GPCPDAY_L3_20150101_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150101_V3.3.nc4 em 0.6s (1.5MB)
Baixando GPCPDAY_L3_20150103_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150103_V3.3.nc4 em 0.6s (1.5MB)
Baixando GPCPDAY_L3_20150102_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150102_V3.3.nc4 em 0.7s (1.5MB)
Baixando GPCPDAY_L3_20150104_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150104_V3.3.nc4 em 0.8s (1.5MB)
Baixando GPCPDAY_L3_20150105_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150105_V3.3.nc4 em 0.8s (1.5MB)
Baixando GPCPDAY_L3_20150106_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150106_V3.3.nc4 em 0.8s (1.5MB)
Baixando GPCPDAY_L3_20150107_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150107_V3.3.nc4 em 0.6s (1.5MB)
Baixando GPCPDAY_L3_20150108_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150108_V3.3.nc4 em 0.8s (1.5MB)
Baixando GPCPDAY_L3_20150110_V3.3.nc4: 1.5MB/1.5MB (100.0%)
[ok] GPCPDAY_L3_20150110_V3.

### Leitura e concatenação de todos nc4 em um único dataframe

In [267]:
import re
import os
import xarray as xr
from pathlib import Path

# -------------------- Config --------------------
FOLDER = "./dados_gpcp"            # pasta com os .nc4 baixados
VARIABLE = None                    
OUTPUT_CSV = "gpcp_daily_concat.csv"
OUTPUT_PARQUET = "gpcp_daily_concat.parquet"

NORMALIZE_LON = True               # converte lon para [-180, 180]
SORT_OUTPUT = True                 # ordena por (date, lat, lon)

#  bounding box do Pará – ative se quiser filtrar já aqui
USE_PA_BBOX = False
PA_BBOX = dict(lon_min=-60.5, lon_max=-46.0, lat_min=-9.8, lat_max=2.5)

# -------------------- Funções utilitárias --------------------
DATE_RE = re.compile(r"(\d{8})")  # captura YYYYMMDD no nome do arquivo

def parse_date_from_name(path: Path) -> np.datetime64:
    m = DATE_RE.search(path.name)
    if not m:
        raise ValueError(f"Não consegui extrair YYYYMMDD do nome: {path.name}")
    ymd = m.group(1)
    return np.datetime64(f"{ymd[:4]}-{ymd[4:6]}-{ymd[6:8]}")

def load_one_dataset(p: Path) -> xr.Dataset:
    ds = xr.open_dataset(p)
    # garante dimensão temporal (muitos diários já têm 'time' com 1 passo)
    if "time" not in ds.coords and "time" not in ds.dims:
        t = parse_date_from_name(p)
        ds = ds.expand_dims(time=[t])
    # assegura que time seja datetime64 ns
    if np.issubdtype(ds["time"].dtype, np.datetime64) is False:
        ds["time"] = xr.decode_cf(ds).get("time", ds["time"])
    return ds

def pick_main_var(ds: xr.Dataset, prefer: str | None = None) -> str:
    if prefer and prefer in ds.data_vars:
        return prefer
    ignore = {"mask", "lsmask", "land_sea_mask", "lat_bnds", "lon_bnds"}
    for v in ds.data_vars:
        if v.lower() in ignore:
            continue
        if ds[v].ndim >= 2:
            return v
    # fallback: a primeira variável qualquer
    return list(ds.data_vars)[0]

# -------------------- Coleta e leitura --------------------
files = sorted(Path(FOLDER).glob("*.nc4"))
if not files:
    raise SystemExit(f"Nenhum .nc4 encontrado em {Path(FOLDER).resolve()}")

datasets = []
failed = []
for i, p in enumerate(files, 1):
    try:
        ds = load_one_dataset(p)
        datasets.append(ds)
        print(f"[{i}/{len(files)}] ok → {p.name}")
    except Exception as e:
        failed.append((p.name, str(e)))
        print(f"[{i}/{len(files)}] erro → {p.name}: {e}")

if not datasets:
    raise SystemExit("Nenhum dataset pôde ser aberto.")



[1/731] ok → GPCPDAY_L3_20150101_V3.3.nc4
[2/731] ok → GPCPDAY_L3_20150102_V3.3.nc4
[3/731] ok → GPCPDAY_L3_20150103_V3.3.nc4
[4/731] ok → GPCPDAY_L3_20150104_V3.3.nc4
[5/731] ok → GPCPDAY_L3_20150105_V3.3.nc4
[6/731] ok → GPCPDAY_L3_20150106_V3.3.nc4
[7/731] ok → GPCPDAY_L3_20150107_V3.3.nc4
[8/731] ok → GPCPDAY_L3_20150108_V3.3.nc4
[9/731] ok → GPCPDAY_L3_20150109_V3.3.nc4
[10/731] ok → GPCPDAY_L3_20150110_V3.3.nc4
[11/731] ok → GPCPDAY_L3_20150111_V3.3.nc4
[12/731] ok → GPCPDAY_L3_20150112_V3.3.nc4
[13/731] ok → GPCPDAY_L3_20150113_V3.3.nc4
[14/731] ok → GPCPDAY_L3_20150114_V3.3.nc4
[15/731] ok → GPCPDAY_L3_20150115_V3.3.nc4
[16/731] ok → GPCPDAY_L3_20150116_V3.3.nc4
[17/731] ok → GPCPDAY_L3_20150117_V3.3.nc4
[18/731] ok → GPCPDAY_L3_20150118_V3.3.nc4
[19/731] ok → GPCPDAY_L3_20150119_V3.3.nc4
[20/731] ok → GPCPDAY_L3_20150120_V3.3.nc4
[21/731] ok → GPCPDAY_L3_20150121_V3.3.nc4
[22/731] ok → GPCPDAY_L3_20150122_V3.3.nc4
[23/731] ok → GPCPDAY_L3_20150123_V3.3.nc4
[24/731] ok → GPCPDA

In [268]:
df_test = datasets[0].to_dataframe()
print(df_test)

                                precip  probability_liquid_phase  time_bnds
time       lat    lon     bnds                                             
2015-01-01  89.75 -179.75 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                  -179.25 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                  -178.75 0        0.0                       0.0 2015-01-01
...                                ...                       ...        ...
           -89.75  178.75 1        0.0                       0.0 2015-01-02
                   179.25 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                   179.75 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02

[518400 row

In [269]:
# -------------------- Concatenação temporal --------------------
# concatena ao longo de 'time' (cada arquivo diário vira 1 passo temporal)
ds_all = xr.concat(datasets, dim="time", join="outer", combine_attrs="override")

# escolhe a variável principal
var_name = pick_main_var(ds_all, prefer=VARIABLE)
print(f"Variável selecionada: {var_name}")

# renomeia para 'precip' (padrão), mantendo apenas o necessário
ds_main = ds_all[[var_name]].rename({var_name: "precip"})


# 1) ative o recorte e use o bbox do Pará (IBGE, extremos oficiais)
USE_PA_BBOX = True
PA_BBOX = dict(
    lon_min=-58.89833,  # Oeste  (−58°53′54″)
    lon_max=-46.06083,  # Leste  (−46°03′39″)
    lat_min=-9.84111,   # Sul    (−09°50′28″)
    lat_max= 2.59111    # Norte  (+02°35′28″)
)

# 2) normaliza longitude p/ [-180, 180] (mantém seu trecho)
if NORMALIZE_LON and ds_main.coords.get("lon") is not None:
    lon = ds_main["lon"]
    if lon.max() > 180:
        ds_main = ds_main.assign_coords(lon=(((lon + 180) % 360) - 180))

# 3) recorte robusto ao sentido das coordenadas
if USE_PA_BBOX and {"lon", "lat"}.issubset(ds_main.coords):
    # lon pode estar crescente (−180→+180) – normalmente está
    lon_vals = ds_main["lon"].values
    lon_slice = slice(PA_BBOX["lon_min"], PA_BBOX["lon_max"]) \
        if lon_vals[0] < lon_vals[-1] else slice(PA_BBOX["lon_max"], PA_BBOX["lon_min"])

    # lat às vezes vem decrescente (+90→−90);
    lat_vals = ds_main["lat"].values
    lat_slice = slice(PA_BBOX["lat_min"], PA_BBOX["lat_max"]) \
        if lat_vals[0] < lat_vals[-1] else slice(PA_BBOX["lat_max"], PA_BBOX["lat_min"])

    ds_main = ds_main.sel(lon=lon_slice, lat=lat_slice)


Variável selecionada: precip


In [270]:

# -------------------- DataFrame final --------------------
# Garante nomes padrão de coord
rename_coords = {}
for cand in ["latitude", "Latitude", "LAT"]:
    if "lat" not in ds_main.coords and cand in ds_main.coords:
        rename_coords[cand] = "lat"
for cand in ["longitude", "Longitude", "LON"]:
    if "lon" not in ds_main.coords and cand in ds_main.coords:
        rename_coords[cand] = "lon"
if rename_coords:
    ds_main = ds_main.rename(rename_coords)

# Para DataFrame "longo": (date, lat, lon, precip)
df = ds_main.to_dataframe().reset_index()

# Renomeia coluna temporal para 'date'
if "time" in df.columns:
    df = df.rename(columns={"time": "date"})

# Colunas em ordem amigável
cols = [c for c in ["date", "lat", "lon", "precip"] if c in df.columns] + \
       [c for c in df.columns if c not in {"date", "lat", "lon", "precip"}]
df_chuva = df[cols]

# Ordena (opcional)
if SORT_OUTPUT:
    sort_cols = [c for c in ["date", "lat", "lon"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols).reset_index(drop=True)


# -------------------- Resumo --------------------
print(f"\n✅ DataFrame criado: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
if "date" in df.columns:
    print(f"   período: {pd.to_datetime(df['date']).min().date()} → {pd.to_datetime(df['date']).max().date()}")
if failed:
    print("\n⚠️ Arquivos com erro:")
    for name, err in failed[:10]:
        print(f" - {name}: {err}")
    if len(failed) > 10:
        print(f"... e mais {len(failed)-10} erros.")


✅ DataFrame criado: 475,150 linhas x 4 colunas
   período: 2015-01-01 → 2016-12-31


### KNN para definir mesorregiões paraenses

In [271]:

USE_PA_BBOX = True
PA_BBOX = dict(lon_min=-58.89833, lon_max=-46.06083,  # Oeste/Leste
               lat_min=-9.84111,  lat_max= 2.59111)   # Sul/Norte

if USE_PA_BBOX:
    df_chuva = df_chuva[
        (df_chuva['lon'] >= PA_BBOX['lon_min']) & (df_chuva['lon'] <= PA_BBOX['lon_max']) &
        (df_chuva['lat'] >= PA_BBOX['lat_min']) & (df_chuva['lat'] <= PA_BBOX['lat_max'])
    ].copy()

# ------------- centróides aproximados das mesorregiões (WGS84) -------------

meso_centroids = pd.DataFrame([
    # name                                lat      lon
    ["Baixo Amazonas (PA)",              -2.40,  -56.40],  # região Santarém/Óbidos/Oriximiná
    ["Marajó (PA)",                      -0.90,  -49.50],  # ilha do Marajó (Soure/Salvaterra)
    ["Metropolitana de Belém (PA)",      -1.45,  -48.50],  # Belém/Ananindeua
    ["Nordeste Paraense (PA)",           -1.10,  -47.30],  # Capanema/Bragança/Castanhal eixo
    ["Sudoeste Paraense (PA)",           -6.00,  -55.00],  # Itaituba/Novo Progresso/Altamira S-O
    ["Sudeste Paraense (PA)",            -5.50,  -49.30],  # Marabá/Parauapebas/Redenção
], columns=["mesoregion","lat_c","lon_c"])

# ---------------- FUNÇÕES ----------------
def haversine_np(lat1, lon1, lat2, lon2):
    """
    Distância Haversine vetorizada (km).
    """
    R = 6371.0088  # raio médio da Terra em km
    # converte para radianos
    lat1_rad = np.radians(lat1); lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2); lon2_rad = np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def assign_mesoregion_knn(df_points, centroids):
    """
    df_points: DataFrame com colunas ['lat','lon']
    centroids: DataFrame com ['mesoregion','lat_c','lon_c']
    Retorna Series com rótulo da mesorregião (string) por linha de df_points.
    """
    # matriz distâncias: (n_points, n_centroids)
    dists = haversine_np(
        df_points['lat'].to_numpy()[:, None],
        df_points['lon'].to_numpy()[:, None],
        centroids['lat_c'].to_numpy()[None, :],
        centroids['lon_c'].to_numpy()[None, :]
    )
    idx_min = np.argmin(dists, axis=1)
    return pd.Series(centroids['mesoregion'].to_numpy()[idx_min], index=df_points.index)


### Agrupando mesoregião para agregar média de precipitação

In [285]:
for col in ['mesoregion', 'mesoregion_x', 'mesoregion_y']:
    if col in df_chuva.columns:
        df_chuva = df_chuva.drop(columns=col)

# mapeia (lat, lon) -> mesoregion sem usar merge (evita conflitos)
coord_to_meso = { (r.lat, r.lon): r.mesoregion for r in coords.itertuples(index=False) }
df_chuva['mesoregion'] = list(map(coord_to_meso.get, zip(df_chuva['lat'], df_chuva['lon'])))

# --- agregação diária por mesorregião ---
df_meso_daily = (
    df_chuva
    .dropna(subset=['mesoregion'])
    .groupby(['date', 'mesoregion'], as_index=False)
    .agg(precip_mean=('precip', 'mean'))
)


### Agrupando semanas para agregar média de precipitação

Aqui também foi necessário criar um "lag" no dado para que as chuvas de semanas atrás possam se relacionar com o período de colheita.

O valor de lag_k e win_sizes foram definidos a partir de dados dessa pesquisa, que menciona um período de 6-9 meses de chuva para explicar a colheita da safra em questão (https://onlinelibrary.wiley.com/doi/10.1002/jsfa.10164?utm_source=chatgpt.com).

In [324]:
df_meso_daily['date'] = pd.to_datetime(df_meso_daily['date'])

# define semana ISO (início na segunda-feira)
week_start = df_meso_daily['date'] - pd.to_timedelta(df_meso_daily['date'].dt.weekday, unit='D')
df_meso_daily['week_start'] = week_start
df_meso_daily['week_end'] = df_meso_daily['week_start'] + pd.Timedelta(days=6)

# acumulado semanal por mesorregião (mm/semana)
df_meso_weekly = (
    df_meso_daily
    .groupby(['mesoregion', 'week_start', 'week_end'], as_index=False)
    .agg(precip_sum_week_mm=('precip_mean', 'sum'))  # 'precip_mean' está em mm/dia → soma = mm/semana
    .sort_values(['mesoregion', 'week_start'])
    .reset_index(drop=True)
)
# --- parâmetros ---
LAG_K = 36       
WIN_SIZES = [1]  

# --- features defasadas (sem vazamento) ---
df_feat = (
    df_meso_weekly
    .sort_values(['mesoregion','week_start'])
    .copy()
)

# lag simples (semana exata K atrás)
df_feat[f'precip_sum_lag_{LAG_K}w'] = (
    df_feat.groupby('mesoregion')['precip_sum_week_mm'].shift(LAG_K)
)

# acumulados: soma das últimas W semanas, terminando K semanas antes
for W in WIN_SIZES:
    df_feat[f'precip_rollsum_{W}w_end_lag_{LAG_K}w'] = (
        df_feat.groupby('mesoregion')['precip_sum_week_mm']
              .apply(lambda s: s.shift(LAG_K).rolling(W, min_periods=1).sum())
              .reset_index(level=0, drop=True)
    )

# mantém só chave + features (evita usar chuva da semana corrente)
df_feat = df_feat[['mesoregion','week_start'] + [c for c in df_feat.columns if c.startswith('precip_')]]

# --- corte dinâmico para evitar NaN por falta de histórico ---
weeks_needed = LAG_K + (max(WIN_SIZES) if WIN_SIZES else 0) - 1
cutoff = df_meso_weekly['week_start'].min() + pd.Timedelta(weeks=max(0, weeks_needed))
df_meso_weekly = df_feat[df_feat['week_start'] >= cutoff].reset_index(drop=True)

cutoff = pd.Timestamp('2016-01-01') 
df_meso_weekly = (
    df_meso_weekly[df_meso_weekly['week_start'] >= cutoff]
    .reset_index(drop=True)
)



In [325]:
print(df_meso_weekly.head())

            mesoregion week_start  precip_sum_week_mm  precip_sum_lag_36w  \
0  Baixo Amazonas (PA) 2016-01-04           10.979014           94.528748   
1  Baixo Amazonas (PA) 2016-01-11           29.736334          102.301491   
2  Baixo Amazonas (PA) 2016-01-18           18.624718           37.129051   
3  Baixo Amazonas (PA) 2016-01-25           47.250248           64.464615   
4  Baixo Amazonas (PA) 2016-02-01           88.343094           28.558430   

   precip_rollsum_1w_end_lag_36w  
0                      94.528748  
1                     102.301491  
2                      37.129051  
3                      64.464615  
4                      28.558430  


In [326]:
acai_dataset = pd.read_excel("./Acai_Datasets/Lavoura_Permanente_Acai_Para.xlsx",  sheet_name='Quantidade produzida (Tonela...',header=None)


## Tratamento dos Dados

 Necessário converter os dados relacionados a açaí para o formato Série Temporal (por semana).

In [327]:
training_data_df = (
    acai_dataset[acai_dataset[0].eq('ME') & acai_dataset[2].notna()][[2, 3]]
      .rename(columns={2: 'mesoregion', 3: 'toneladas'})
)

# normaliza tipos/strings
training_data_df['mesoregion'] = training_data_df['mesoregion'].astype(str).str.strip()
training_data_df['toneladas'] = pd.to_numeric(training_data_df['toneladas'], errors='coerce')

# adiciona ano e reordena
training_data_df.insert(1, 'ano', 2016)
training_data_df = training_data_df[['mesoregion', 'ano', 'toneladas']]

training_data_df

,mesoregion,ano,toneladas
12,Baixo Amazonas (PA),2016,9834
13,Marajó (PA),2016,152465
14,Metropolitana de Belém (PA),2016,158930
15,Nordeste Paraense (PA),2016,700842
16,Sudoeste Paraense (PA),2016,3061
17,Sudeste Paraense (PA),2016,55480


Agora os dados serão convertidos para um modelo de semana em semana, utilizando como referência os seguintes valores de distribuição de produção (em toneladas)

- Mensal (BRS Pará, sem irrigação): Jan 15, Fev 10, Mar 3, Abr 2, Mai 0, Jun 0, Jul 0, Ago 3, Set 7, Out 15, Nov 25, Dez 20. (https://www.infoteca.cnptia.embrapa.br/infoteca/bitstream/doc/1101575/1/SistemadeproducaoAcai2018.pdf)
- Semestres (Pará, sem irrigação): S1 (jan–jun) ~20–30% · S2 (jul–dez) ~70–80%.  https://www.alice.cnptia.embrapa.br/bitstream/doc/994953/1/CULTIVO20.pdf

In [328]:
MONTH_WEIGHTS = {1:0.15,2:0.10,3:0.03,4:0.02,5:0.00,6:0.00,7:0.00,8:0.03,9:0.07,10:0.15,11:0.25,12:0.20}

def mondays_in_month(year: int, month: int) -> pd.DatetimeIndex:
    start = pd.Timestamp(year=year, month=month, day=1)
    end = pd.Timestamp(year=year, month=month, day=monthrange(year, month)[1])
    d = pd.date_range(start, end, freq="W-MON")
    if start.weekday() == 0 and (len(d) == 0 or d[0] != start):
        d = d.insert(0, start)
    return d[d.month == month]

def generate_weekly_series_meso(df_meso: pd.DataFrame, seed: int = 42) -> pd.DataFrame:
    """
    df_meso: colunas ['mesoregion','ano','toneladas'] (um total anual por mesorregião)
    retorna: semanal com ['mesoregion','date','year','month','week','toneladas_semana','dist_ano','dist_mes','periodo']
    """
    rng = np.random.default_rng(seed)
    if not np.isclose(sum(MONTH_WEIGHTS[m] for m in range(1,7)), 0.30) or \
       not np.isclose(sum(MONTH_WEIGHTS[m] for m in range(7,13)), 0.70):
        raise ValueError("MONTH_WEIGHTS deve somar 30% (jan-jun) e 70% (jul-dez).")

    rows = []
    for _, r in df_meso.iterrows():
        reg  = str(r['mesoregion'])
        year = int(r['ano'])
        total_year = float(r['toneladas'])

        for m in range(1,13):
            w = MONTH_WEIGHTS[m]
            month_total = total_year * w
            weeks = mondays_in_month(year, m)
            n = len(weeks)
            if n == 0: 
                continue
            alpha = 5.0
            shares = np.zeros(n) if w == 0.0 else rng.dirichlet(np.full(n, alpha))
            tons = month_total * shares
            for d, t in zip(weeks, tons):
                rows.append({
                    'mesoregion': reg,
                    'date': d.normalize(),
                    'year': year,
                    'month': m,
                    'week': int(d.isocalendar().week),
                    'toneladas_semana': float(t),
                    'dist_ano': float(t / total_year if total_year > 0 else 0.0),
                    'dist_mes': float(t / month_total) if month_total > 0 else (0.0 if w == 0 else np.nan),
                    'periodo': 'entressafra' if m <= 6 else 'safra'
                })
    return pd.DataFrame(rows).sort_values(['mesoregion','year','date']).reset_index(drop=True)


df_meso_annual = pd.DataFrame({
     'mesoregion': ['Baixo Amazonas (PA)','Marajó (PA)','Metropolitana de Belém (PA)','Nordeste Paraense (PA)','Sudoeste Paraense (PA)','Sudeste Paraense (PA)'],
     'ano': [2016]*6,
     'toneladas': [9834, 152465, 158930, 700842, 3061, 55480] })
weekly_series_meso = generate_weekly_series_meso(df_meso_annual, seed=123)
weekly_series_meso.head(80)

,mesoregion,date,year,month,week,toneladas_semana,dist_ano,dist_mes,periodo
0,Baixo Amazonas (PA),2016-01-04,2016,1,1,197.100086,0.020043,0.133618,entressafra
1,Baixo Amazonas (PA),2016-01-11,2016,1,2,557.988023,0.056741,0.378271,entressafra
2,Baixo Amazonas (PA),2016-01-18,2016,1,3,482.449373,0.049059,0.327062,entressafra
3,Baixo Amazonas (PA),2016-01-25,2016,1,4,237.562519,0.024157,0.161048,entressafra
4,Baixo Amazonas (PA),2016-02-01,2016,2,5,120.990118,0.012303,0.123032,entressafra
...,...,...,...,...,...,...,...,...,...
75,Marajó (PA),2016-06-13,2016,6,24,0.000000,0.000000,0.000000,entressafra
76,Marajó (PA),2016-06-20,2016,6,25,0.000000,0.000000,0.000000,entressafra
77,Marajó (PA),2016-06-27,2016,6,26,0.000000,0.000000,0.000000,entressafra
78,Marajó (PA),2016-07-04,2016,7,27,0.000000,0.000000,0.000000,safra


### Concatenação dos dados de precipitação e dos dados de produção

In [329]:

# 1) chave semanal = segunda-feira (week_start)
df_meso_weekly['week_start'] = pd.to_datetime(df_meso_weekly['week_start'])
df_meso_weekly['mesoregion'] = df_meso_weekly['mesoregion'].astype(str).str.strip()

weekly_series_meso['date'] = pd.to_datetime(weekly_series_meso['date'])
weekly_series_meso['mesoregion'] = weekly_series_meso['mesoregion'].astype(str).str.strip()
wk_2016 = weekly_series_meso[weekly_series_meso['year'] == 2016].copy()
wk_2016 = wk_2016.rename(columns={'date':'week_start'})


# 2) merge limpo por mesorregião + semana (agora levando TODAS as colunas precip_ de df_meso_weekly)
precip_cols = [c for c in df_meso_weekly.columns if c.startswith('precip_')]
df_weekly_join = (
    wk_2016.merge(
        df_meso_weekly[['mesoregion','week_start'] + precip_cols],
        on=['mesoregion','week_start'],
        how='left',
        validate='one_to_one'
    )
    .sort_values(['mesoregion','week_start'])
    .reset_index(drop=True)
)

df_weekly_join['week_end'] = df_weekly_join['week_start'] + pd.Timedelta(days=6)


base_cols = ['mesoregion','week_start','week_end','year','month','week',
             'toneladas_semana','periodo']
training_data = df_weekly_join[base_cols + precip_cols]


#training_data = training_data[training_data['mesoregion'] == 'Nordeste Paraense (PA)']
training_data 

,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_1w_end_lag_36w
0,Baixo Amazonas (PA),2016-01-04,2016-01-10,2016,1,1,197.100086,entressafra,10.979014,94.528748,94.528748
1,Baixo Amazonas (PA),2016-01-11,2016-01-17,2016,1,2,557.988023,entressafra,29.736334,102.301491,102.301491
2,Baixo Amazonas (PA),2016-01-18,2016-01-24,2016,1,3,482.449373,entressafra,18.624718,37.129051,37.129051
3,Baixo Amazonas (PA),2016-01-25,2016-01-31,2016,1,4,237.562519,entressafra,47.250248,64.464615,64.464615
4,Baixo Amazonas (PA),2016-02-01,2016-02-07,2016,2,5,120.990118,entressafra,88.343094,28.558430,28.558430
...,...,...,...,...,...,...,...,...,...,...,...
307,Sudoeste Paraense (PA),2016-11-28,2016-12-04,2016,11,48,360.530187,safra,41.873493,64.160782,64.160782
308,Sudoeste Paraense (PA),2016-12-05,2016-12-11,2016,12,49,109.671780,safra,48.261589,70.456482,70.456482
309,Sudoeste Paraense (PA),2016-12-12,2016-12-18,2016,12,50,287.955953,safra,93.745483,30.292412,30.292412
310,Sudoeste Paraense (PA),2016-12-19,2016-12-25,2016,12,51,104.625441,safra,83.633514,58.702126,58.702126


## Análise dos Dados

É possível aferir sobre a precipitação observando a média, máxima e mediana, que existe uma forte assimetria, destacando que algumas semanas são responsáveis por boa parte da precipitação do trimestre.


In [330]:
training_data.describe(include="all")

,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_1w_end_lag_36w
count,312,312,312,312.0,312.000000,312.000000,312.000000,312,312.000000,312.000000,312.000000
unique,6,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
top,Baixo Amazonas (PA),NaN,NaN,NaN,NaN,NaN,NaN,entressafra,NaN,NaN,NaN
freq,52,NaN,NaN,NaN,NaN,NaN,NaN,156,NaN,NaN,NaN
mean,NaN,2016-06-30 12:00:00,2016-07-06 12:00:00,2016.0,6.480769,26.500000,3463.500000,NaN,39.155708,35.347473,35.347476
min,NaN,2016-01-04 00:00:00,2016-01-10 00:00:00,2016.0,1.000000,1.000000,0.000000,NaN,0.002183,0.009501,0.009501
25%,NaN,2016-04-02 06:00:00,2016-04-08 06:00:00,2016.0,3.750000,13.750000,5.042830,NaN,10.627479,6.974639,6.974639
50%,NaN,2016-06-30 12:00:00,2016-07-06 12:00:00,2016.0,6.500000,26.500000,383.195702,NaN,27.197410,20.361839,20.361839
75%,NaN,2016-09-27 18:00:00,2016-10-03 18:00:00,2016.0,9.250000,39.250000,3157.810974,NaN,59.220059,57.027666,57.027666
max,NaN,2016-12-26 00:00:00,2017-01-01 00:00:00,2016.0,12.000000,52.000000,66932.916495,NaN,165.539368,154.025330,154.025330


In [331]:
training_data.corr(numeric_only = True)

,year,month,week,toneladas_semana,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_1w_end_lag_36w
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month,NaN,1.000000,0.996531,0.196714,-0.406494,0.418577,0.418577
week,NaN,0.996531,1.000000,0.197227,-0.402858,0.410389,0.410389
toneladas_semana,NaN,0.196714,0.197227,1.000000,-0.099059,0.356725,0.356725
precip_sum_week_mm,NaN,-0.406494,-0.402858,-0.099059,1.000000,-0.044859,-0.044859
precip_sum_lag_36w,NaN,0.418577,0.410389,0.356725,-0.044859,1.000000,1.000000
precip_rollsum_1w_end_lag_36w,NaN,0.418577,0.410389,0.356725,-0.044859,1.000000,1.000000


Maior quantidade de chuvas no começo e final do ano, uma menor quatidade de chuva ao meio do ano, ainda que dificilmente chegue em 0 de fato.

In [332]:
fig = px.scatter_matrix(
    training_data,
    dimensions=['precip_sum_week_mm', 'week'],
    color='mesoregion'
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

Existe uma maior produção de açaí no começo e final do ano. Meio do ano costuma apresentar menos açaí.

In [333]:
fig = px.scatter_matrix(
    training_data,
    dimensions=['toneladas_semana', 'week'],
    color='mesoregion'
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

É interessante notar que mais chuvas não significa necessariamente mais toneladas a serem produzidas, e sim que existe uma certa quantidade de chuva ideal para a produção de açaí.

Como existem muitos outliers, provavelmente o MSE será a fórmula utilizada para métrica de erros.

In [334]:

fig = px.scatter_matrix(
    training_data,
    dimensions=['precip_sum_lag_36w', 'toneladas_semana'],
    color='mesoregion'
)

fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()